# June 15, 2024: create roi timeseries, runwise

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from multiprocessing import Pool
import glob

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

In [3]:
args.type = 'spatial'
args.roi_size = 225
args.maintain_symmetry = True
args.brain_div = 'whl'
args.num_rois = 162

PARC_DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)

BASE_path = f'{os.environ["HOME"]}/mouse_dataset/roi_results_v2'
ROI_path = f'{BASE_path}/{PARC_DESC}'
TS_path = f'{ROI_path}/runwise_timeseries'
os.system(f'mkdir -p {TS_path}')

0

In [4]:
os.system(
    f'bash 01a-desc-run-runwise-roi-timeseries.sh {PARC_DESC} {TS_path}'
)

sub-SLC01_ses-2_run-1_task-rest_desc-roi-ts


++ 3dcalc: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
++ Authored by: A cast of thousands
** AFNI converts NIFTI_datatype=64 (FLOAT64) in file /mnt/stor1/govindas/mouse_dataset/voxel/cleaned_timeseries/_split_name_sub-SLC01_ses-2_task-rest_run-1_bold/sub-SLC01_ses-2_task-rest_run-1_bold_combined_cleaned.nii.gz to FLOAT32
     Warnings of this type will be muted for this session.
     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
++ Output dataset /home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/runwise_timeseries/sub-SLC01_ses-2_run-1_task-rest_desc-roi-ts.nii.gz


sub-SLC01_ses-3_run-3_task-rest_desc-roi-ts
sub-SLC01_ses-1_run-3_task-rest_desc-roi-ts


++ 3dcalc: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
++ Authored by: A cast of thousands
** AFNI converts NIFTI_datatype=64 (FLOAT64) in file /mnt/stor1/govindas/mouse_dataset/voxel/cleaned_timeseries/_split_name_sub-SLC01_ses-3_task-rest_run-3_bold/sub-SLC01_ses-3_task-rest_run-3_bold_combined_cleaned.nii.gz to FLOAT32
     Warnings of this type will be muted for this session.
     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
++ Output dataset /home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/runwise_timeseries/sub-SLC01_ses-3_run-3_task-rest_desc-roi-ts.nii.gz
++ 3dcalc: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
++ Authored by: A cast of thousands
** AFNI converts NIFTI_datatype=64 (FLOAT64) in file /mnt/stor1/govindas/mouse_dataset/voxel/cleaned_timeseries/_split_name_sub-SLC01_ses-1_task-rest_run-3_bold/sub-SLC01_ses-1_task-rest_run-3_bold_combined_cleaned.nii.gz to FLOAT32
     Warnings of this type

sub-SLC01_ses-2_run-7_task-rest_desc-roi-ts


...

sub-SLC10_ses-3_run-3_task-rest_desc-roi-ts


++ 3dcalc: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
++ Authored by: A cast of thousands
** AFNI converts NIFTI_datatype=64 (FLOAT64) in file /mnt/stor1/govindas/mouse_dataset/voxel/cleaned_timeseries/_split_name_sub-SLC10_ses-3_task-rest_run-3_bold/sub-SLC10_ses-3_task-rest_run-3_bold_combined_cleaned.nii.gz to FLOAT32
     Warnings of this type will be muted for this session.
     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
++ Output dataset /home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/runwise_timeseries/sub-SLC10_ses-3_run-3_task-rest_desc-roi-ts.nii.gz


sub-SLC10_ses-3_run-7_task-rest_desc-roi-ts
sub-SLC10_ses-3_run-5_task-rest_desc-roi-ts


++ 3dcalc: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
++ Authored by: A cast of thousands
** AFNI converts NIFTI_datatype=64 (FLOAT64) in file /mnt/stor1/govindas/mouse_dataset/voxel/cleaned_timeseries/_split_name_sub-SLC10_ses-3_task-rest_run-7_bold/sub-SLC10_ses-3_task-rest_run-7_bold_combined_cleaned.nii.gz to FLOAT32
     Warnings of this type will be muted for this session.
     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
++ Output dataset /home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/runwise_timeseries/sub-SLC10_ses-3_run-7_task-rest_desc-roi-ts.nii.gz
++ 3dcalc: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
++ Authored by: A cast of thousands
** AFNI converts NIFTI_datatype=64 (FLOAT64) in file /mnt/stor1/govindas/mouse_dataset/voxel/cleaned_timeseries/_split_name_sub-SLC10_ses-3_task-rest_run-5_bold/sub-SLC10_ses-3_task-rest_run-5_bold_combined_cleaned.nii.gz to FLOAT32
     Warnings of this type

0